In [2]:
import os
import pandas as pd
from pybis import Openbis

# Read Word Files

In [44]:
def read_word_files_from_folder(folderpath): 
    """
    read_word_files_from_folder(string) -> dictionary
    Read MS Word Files - only works for doc files! - no docx"""
    import win32com.client
    word = win32com.client.Dispatch("Word.Application")
    word.visible = False
    filelist = os.listdir(folderpath)
    titles = dict()
    for files in filelist:
        if files[-1:][0][0] =='~':
            next
        if files[-4:] =='.doc':
            file  = os.path.join(folderpath, files)
            wb = word.Documents.Open(file)
            doc = word.ActiveDocument
            document = doc.Range().Text
            titles[files] = extract_title(document, doc)
        else: pass
    word.ActiveDocument.Close()
    return titles

In [58]:
def extract_title(document, doc):
    """Use this to specifically extract titles from Files which have the structure Titel: inside them"""
    titlef = (str(document).replace('\r', ' ').find('Titel:'))
    stich = (str(document).replace('\r', ' ').find('Stichworte:'))
    f = str(doc.Range().Text).replace('\r', ' ')
    title = (f[(titlef+6):(stich)])
    title = ''.join(c for c in title if c.isprintable()).strip()
    return (document, title)

In [48]:
def transform_docx_to_pdf(filepath):
    from docx2pdf import convert
    return convert(filepath)

# Supplier interactions

In [ ]:
def get_suppliers():
    """"Get a list of suppliers and their codes"""
    suppliers = o.get_collection('/STOCK_CATALOG/SUPPLIERS/SUPPLIER_COLLECTION')
    supdict= dict()
    for sups in suppliers.get_samples():
        ids = o.get_sample(sups.identifier).permId
        codes = o.get_sample(sups.identifier).code
        names = o.get_sample(sups.identifier).p('$name')
        supdict[names] = ids, codes
    return suplist

In [ ]:
def new_supplier(CODE, NAME, ADDRESS1, ADDRESS2, PHONE, EMAIL, LANG='English', URL=None):
    """Create a new supplier object on openBIS"""
    sample = o.new_sample(
        type       = 'SUPPLIER',
        experiment = '/STOCK_CATALOG/SUPPLIERS/SUPPLIER_COLLECTION',
        code = CODE,
        props      = {"$name": NAME * (not(pd.isna(NAME))*1),
                     "$supplier.company_address_line_1" : ADDRESS1 * (not(pd.isna(ADDRESS1))*1),
                     "$supplier.company_address_line_2" : ADDRESS2 * (not(pd.isna(ADDRESS2))*1),
                     "$supplier.company_phone": PHONE * (not(pd.isna(PHONE))*1),
                     "$supplier.company_email": EMAIL * (not(pd.isna(EMAIL))*1),
                     "$supplier.company_language": LANG * (not(pd.isna(LANG))*1),
                     "supplier.url" : URL}
    )
    sample.save()